🎓 **Authors:** [Merve Ayyüce KIZRAK](http://www.ayyucekizrak.com/) and [Bülent BOLAT](http://www.yildiz.edu.tr/~bbolat/)

🏢 **Institutions:** [Bahcesehir University](https://bau.edu.tr/) and [Yildiz Technical University](http://www.yildiz.edu.tr/)

📚 **Publisher:** [International Journal of Informatics Technologies](http://dergipark.gov.tr/gazibtd)

---

###  📑 MAKALENİN TAMAMINI OKUMAK İÇİN [BURAYA]() TIKLAYINIZ!

---

## Predictive Maintenance of Aircraft Motor Health with Long-Short Term Memory Method

**Abstract**— Predictive maintenance is an important part of applications that require costly engine maintenance, such as automotive, aircraft and factory automation. It is important to anticipate the maintenance periods of the engines and develop a business management strategy accordingly in terms of both work safety and efficiency. For predictive maintenance, the sensor data from the motors were used to determine the wear time and level of the engine. In this study, a solution based on deep learning is proposed as an alternative to traditional regression and classification methods. The NASA Turbofan Engine Corruption Simulation data set was studied by using Long-Short Term Memory (LSTM), one of the deep learning models and known to make successful predictions on time-dependent data such as time series. During the simulations, the highest classification performance and the lowest mean absolute error were obtained by LSTM as 98,876% 1.343 respectively.


---
## Uçak Motoru Sağlığı için Uzun-Kısa Süreli Bellek Yöntemi ile Öngörücü Bakım

**Özet**—Otomotiv, uçak ve fabrika otomasyonu gibi özellikle maliyetli motor bakımı gerektiren uygulamalarda öngörücü bakım önemli bir yer almaktadır. Hem iş güvenliği hem de araçlardan sağlanacak verim bakımından motorların bakım periyotlarını önceden kestirmek ve buna göre iş yönetim stratejisi geliştirmek önemlidir. Öngörücü bakım için motorlardan alınan sensör verileri motorun yıpranma süresini ve seviyesini belirlemekte kullanılmaktadır. Çalışmada Uzun-Kısa Süreli Bellek (LSTM) yapısı kullanılarak, uçak motorlarının kalan yaşam ömürlerinin tahmin edilmesi amaçlanmıştır. NASA tarafından sunulmuş olan bir veri kümesi üzerinde LSTM yapısı test edilmiştir ve elde edilen sonuçlar farklı yöntemlerle kıyaslanmıştır. Yapılan uygulamaların sonucunda en yüksek sınıflandırma başarımı %98,876; en düşük ortalama mutlak hata ise 1,343 olarak LSTM ile elde edilmiştir.


![PM Görsel](https://drive.google.com/uc?export=view&id=1j7j9Lb5JexEGhpbmAHYuvFjNpHQPs7rG)


---






### **Gerekli kütüphanelerin kurulumu**


---

### **Installation of the necessary frameworks and libraries**

In [0]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability 
# Çoğaltılabilirlik için seed ayarlama
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

Using TensorFlow backend.


### Verileri Yükleme İşlemi - Azure'dan veri kümelerini okuma


---

### Data Ingestion - Reading the Datasets from Azure Blob

In [0]:

!wget http://azuremlsamples.azureml.net/templatedata/PM_train.txt 
!wget http://azuremlsamples.azureml.net/templatedata/PM_test.txt
!wget http://azuremlsamples.azureml.net/templatedata/PM_truth.txt

--2019-04-16 20:19:53--  http://azuremlsamples.azureml.net/templatedata/PM_train.txt
Resolving azuremlsamples.azureml.net (azuremlsamples.azureml.net)... 13.107.246.10, 2620:1ec:bdf::10
Connecting to azuremlsamples.azureml.net (azuremlsamples.azureml.net)|13.107.246.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3515356 (3.4M) [text/plain]
Saving to: ‘PM_train.txt’

PM_train.txt        100%[===================>]   3.35M  3.23MB/s    in 1.0s    

2019-04-16 20:19:55 (3.23 MB/s) - ‘PM_train.txt’ saved [3515356/3515356]

--2019-04-16 20:19:57--  http://azuremlsamples.azureml.net/templatedata/PM_test.txt
Resolving azuremlsamples.azureml.net (azuremlsamples.azureml.net)... 13.107.246.10, 2620:1ec:bdf::10
Connecting to azuremlsamples.azureml.net (azuremlsamples.azureml.net)|13.107.246.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2228855 (2.1M) [text/plain]
Saving to: ‘PM_test.txt’

PM_test.txt         100%[===================>]   2.

**Eğitim Verilerinin Okunması**

---

**Read Training Data**

In [0]:
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

**Test Verilerinin Okunması**

---

**Read Test Data**

In [0]:
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

**Temel Gerçek Verilerin Okunması**

---

**Read Ground Truth Data**

In [0]:
truth_df = pd.read_csv('PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

**Eğitim Veri Kümesinin İlk 5 Değerinden Örnekleri Ekrana Yazdırma**

---

**Shown First 5 Rows From Training Dataset**

In [0]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


**Data Etiketleme - RUL sütunu oluşturma**

---

**Data Labeling - generate column RUL**

In [0]:
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


**Eğitim Verileri için Etiket Sütunları Oluştur**

---

**Generate Label Columns for Training Data**

In [0]:
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0


**Min-Max Normalizasyon**

---
**Min-Max Normalization**


In [0]:
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [0]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108


**Test Verisi için Max içeren Sütun Oluştur**

---

**Generate Column Max for Test Data**

In [0]:
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

**Test Verisi için RUL içeren Sütun Oluştur**

---

**Generate Column RUL for Test Data**

In [0]:
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138


**Test Verisi için w0 ve w1 Etiket Sütunlarını Oluştur**

---

**Generate Label Columns w0 and w1 for Test Data**

In [0]:
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138,0,0


**50 Döngü için Büyük Bir Pencere Seç**

---

**Pick a Large Window Size of 50 Cycles** 

In [0]:
sequence_length = 50

**Data Görselleştirme için Ön Hazırlıklar** - motor kimliği 3 için bir arıza noktasından önceki 50 devir sayısı penceresi

---

**Preparing Data for Visualizations** - window of 50 cycles prior to a failure point for engine id 3

In [0]:
engine_id3 = test_df[test_df['id'] == 3]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
cols1 = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
cols2 = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

**Özellikleri Yeniden Biçimlendirme Fonksiyonu** (örnekler, zaman adımları, özellikler)

Yalnızca pencere uzunluğunu karşılayan diziler dikkate alınır, dolgu kullanılmaz. Bu, test için pencere uzunluğunun altındakileri düşürmemiz gerektiği anlamına gelir.

Bir alternatif, daha kısa olanları kullanabilmemiz için diziler doldurmak olacaktır.

---

**Function to reshape features into (samples, time steps, features)**
Only sequences that meet the window-length are considered, no padding is used. This means for testing we need to drop those which are below the window-length. 

An alternative would be to pad sequences so that we can use shorter ones.

In [0]:
 def gen_sequence(id_df, seq_length, seq_cols):
  
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

**Bir Öznitelik Sütunu Seç**

---

**Pick the Feature Columns**

In [0]:
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

**Dizi Üretme**

---

**Generator for the Sequences**

In [0]:
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

**Dizileri Oluştur ve Numpy Dizisine Dönüştür**

---

**Generate Sequences and Convert to Numpy Array**

In [0]:
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

**Etiket Üretme Fonksiyonu**

---

**Function to Generate Labels**

In [0]:
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

**Etiketleri Üretilmesi**

---

**Generate Labels**

In [0]:
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

## Uzun-Kısa Süreli (Vadeli) Bellek Ağının Oluşturulması  (Derin Öğrenme Modeli)

---

## Build the Long-Short Term Memory Network (Deep Learning Model)

In [0]:
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))

model.add(LSTM(
          units=75,
          return_sequences=False))
model.add(Dropout(0.50))


model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           50400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 75)                52800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 76        
Total params: 183,676
Trainable params: 183,676
Non-trainable params: 0
_______________

**Oluştutulan Ağın Eğitilmesi**

---

**Fit the Network**

In [0]:
%%time 
###
model.fit(seq_array, label_array, epochs=10, batch_size=100, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Train on 14849 samples, validate on 782 samples
Epoch 1/10
14849/14849 [==============================] - 35s 2ms/step - loss: 0.0645 - acc: 0.9721 - val_loss: 0.0650 - val_acc: 0.9757
Epoch 2/10
14849/14849 [==============================] - 35s 2ms/step - loss: 0.0641 - acc: 0.9731 - val_loss: 0.0354 - val_acc: 0.9898
Epoch 3/10
14849/14849 [==============================] - 36s 2ms/step - loss: 0.0588 - acc: 0.9756 - val_loss: 0.0379 - val_acc: 0.9859
CPU times: user 2min 23s, sys: 9.19 s, total: 2min 33s
Wall time: 1min 46s


**Eğitim Metrikleri**

---

**Training Metrics**

In [0]:
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=100)
print('Accurracy: {}'.format(scores[1]))

15631/15631 [==============================] - 14s 873us/step
Accurracy: 0.9809353264346247


**Bir Kestirim Yap ve Confusion Matrisini Hesapla**

---

**Make Predictions and Compute Confusion Matrix**

In [0]:
y_pred = model.predict_classes(seq_array,verbose=1, batch_size=100)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

15631/15631 [==============================] - 14s 868us/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12357,   174],
       [  124,  2976]])

**Hassaslık ve Hatırlatma Değerlerini Hesapla**

---

**Compute Precision and Recall **

In [0]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.9447619047619048 
 recall =  0.96


In [0]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(93, 50, 25)

In [0]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [0]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(93, 1)

In [0]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(93, 50, 25)
(93, 1)


**Test Metrikleri**

---

**Test Metrics**

In [0]:
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

Accurracy: 0.9784946178877225


**Bir Kestirim Yap ve Confusion Matrisini Hesapla**

---

**Make Predictions and Compute Confusion Matrix**

In [0]:
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[67,  1],
       [ 1, 24]])